# Lab 6 - CART and Random Forest Predictions

In this final part of the project, you will build and validate both CART and Random Forest models for predicting each target (Secchi Depth and Total Phosphorus) using the parcel features.

## Tasks

### Data Splitting Strategy

We want to extract 30% of the lakes to be used as a validation set, with the remaining lakes separated into a train/test split (again use 70%/30% split). In summary we should have a random split with:

- **30% of the lakes** marked as "Validation"
- **50% of the lakes** (~70% of 70%) marked as "Training"
- **20% of the lakes** (~30% of 70%) marked as "Test"

### Model Building and Validation

For each target:

1. Perform a grid search for both CART and Random Forests using the training lakes
2. Compare the best model of each type on the test lakes to determine the best overall model
3. Refit the best model on the 70% of the lakes not in the validation set (training + test)
4. Use the validation set to estimate the performance of this model
5. Write up a summary of what we learn

## Step 1: Load and Prepare Data

### Task 1.1: Import Libraries and Load Data

In [35]:
# Import required libraries
%pip install pyarrow
import polars as pl
import polars.selectors as cs
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import pyarrow as pa

Note: you may need to restart the kernel to use updated packages.


In [36]:
(lake_year_data := pl.read_csv('./data/final_dataset_with_parcel_features.csv'))

DNR_ID_Site_Number,LAKE_NAME,Year,latitude,longitude,avg_secchi_depth,avg_total_phosphorus,parcel_count_within_500m,parcel_count_between_501_1600m,parcel_count_over_1600m,emv_land_mean_within_500m,emv_land_mean_between_501_1600m,emv_land_mean_over_1600m,emv_bldg_mean_within_500m,emv_bldg_mean_between_501_1600m,emv_bldg_mean_over_1600m,emv_total_mean_within_500m,emv_total_mean_between_501_1600m,emv_total_mean_over_1600m,acres_poly_mean_within_500m,acres_poly_mean_between_501_1600m,acres_poly_mean_over_1600m,acres_deed_mean_within_500m,acres_deed_mean_between_501_1600m,acres_deed_mean_over_1600m,fin_sq_ft_mean_within_500m,fin_sq_ft_mean_between_501_1600m,fin_sq_ft_mean_over_1600m,year_built_mean_within_500m,year_built_mean_between_501_1600m,year_built_mean_over_1600m,num_units_mean_within_500m,num_units_mean_between_501_1600m,num_units_mean_over_1600m,total_tax_mean_within_500m,total_tax_mean_between_501_1600m,total_tax_mean_over_1600m,…,year_built_median_between_501_1600m,year_built_median_over_1600m,total_tax_median_within_500m,total_tax_median_between_501_1600m,total_tax_median_over_1600m,emv_land_std_within_500m,emv_land_std_between_501_1600m,emv_land_std_over_1600m,emv_bldg_std_within_500m,emv_bldg_std_between_501_1600m,emv_bldg_std_over_1600m,emv_total_std_within_500m,emv_total_std_between_501_1600m,emv_total_std_over_1600m,acres_poly_std_within_500m,acres_poly_std_between_501_1600m,acres_poly_std_over_1600m,num_units_total_within_500m,num_units_total_between_501_1600m,num_units_total_over_1600m,basement_prop_within_500m,basement_prop_between_501_1600m,basement_prop_over_1600m,garage_prop_within_500m,garage_prop_between_501_1600m,garage_prop_over_1600m,green_acre_prop_within_500m,green_acre_prop_between_501_1600m,green_acre_prop_over_1600m,open_space_prop_within_500m,open_space_prop_between_501_1600m,open_space_prop_over_1600m,ag_preserv_prop_within_500m,ag_preserv_prop_between_501_1600m,ag_preserv_prop_over_1600m,centroid_lat,centroid_long
str,str,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""82007700-01""","""Goggins Lake""",2006,45.133035,-92.892832,0.956,0.102,69,137,9,204921.73913,171323.357664,151266.666667,147011.594203,199991.240876,230944.444444,351933.333333,371314.59854,382211.111111,5.32913,4.568102,3.212222,10.593043,8.489343,6.68,1234.115942,1503.124088,1555.666667,1369.971014,1502.131387,1526.777778,0.0,0.0,0.0,2050.202899,2182.788321,2745.111111,…,1981.0,1978.0,2366.0,2424.0,2884.0,128945.44089,102820.380406,87242.635792,121999.144809,193184.768868,275852.316064,195013.07925,228000.633558,316231.001186,9.497451,8.09445,4.598836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.202899,0.116788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.12616,-92.8821
"""19002700-01""","""Crystal Lake""",2014,44.722968,-93.270366,1.96,0.0244,1765,92,null,110654.787535,84493.478261,null,159819.320113,189930.434783,null,270474.107649,274423.913043,null,0.430295,0.606739,null,0.0,0.0,null,2228.563173,2573.119565,null,1821.904816,1872.815217,null,0.92068,0.923913,null,3317.18187,4132.423913,null,…,1979.0,null,2754.0,2792.5,null,138105.254957,120359.683298,null,82338.486478,129628.188275,null,168611.758605,230124.674938,null,0.769999,1.731464,null,1625.0,85.0,null,0.0,0.0,null,0.0,0.0,null,0.0,0.0,null,0.003399,0.0,null,0.0,0.0,null,44.71658,-93.26527
"""82012200-01""","""Pine Tree Lake""",2004,45.102314,-92.953869,2.2625,0.027375,273,424,null,173076.556777,160262.971698,null,301198.168498,319631.839623,null,474274.725275,479894.811321,null,2.584652,2.91342,null,5.146777,6.124481,null,0.0,0.0,null,0.0,0.0,null,0.0,0.0,null,3770.131868,3905.820755,null,…,0.0,null,3632.0,3458.0,null,111947.950582,135619.676884,null,332707.974039,260368.938186,null,398363.670937,309997.671648,null,7.763072,10.417772,null,0.0,0.0,nu

In [37]:
# Split lakes: 30% validation, 50% training, 20% test
np.random.seed(42)
(unique_lakes := lake_year_data['DNR_ID_Site_Number'].unique().to_list())
np.random.shuffle(unique_lakes)

(val_split := int(0.30 * len(unique_lakes)))
(train_split := val_split + int(0.5 * len(unique_lakes)))

(lake_year_data := lake_year_data.with_columns(
    pl.when(pl.col('DNR_ID_Site_Number').is_in(unique_lakes[:val_split]))
      .then(pl.lit('Validation'))
    .when(pl.col('DNR_ID_Site_Number').is_in(unique_lakes[val_split:train_split]))
      .then(pl.lit('Training'))
    .otherwise(pl.lit('Test'))
    .alias('split')
))

(lake_year_data
 .group_by('split')
 .agg(pl.len().alias('rows'))
 .with_columns(
     (pl.col('rows') / pl.col('rows').sum()).alias('proportion')
 )
 .sort('split')
)

split,rows,proportion
str,u32,f64
"""Test""",110,0.208333
"""Training""",264,0.5
"""Validation""",154,0.291667


### Train Test Split

In [38]:
# Identify columns to exclude (IDs, target variables, split column, and any string columns)
exclude_cols = ['DNR_ID_Site_Number', 'Monit_MAP_CODE1', 'Year', 
                'avg_secchi_depth', 'avg_total_phosphorus', 'Avg_Secchi', 'Avg_Total_Phosphorus',
                'Lake_Name', 'lake_name', 'centroid_lat', 'centroid_long', 'split']

# Get feature columns (only numeric columns)
feature_cols = [col for col in lake_year_data.columns 
                if col not in exclude_cols and lake_year_data[col].dtype in [pl.Int64, pl.Float64]]

# Split into training+test (70%) and validation (30%)
(train_test_df := lake_year_data.filter(pl.col('split').is_in(['Training', 'Test'])))
(validation_df := lake_year_data.filter(pl.col('split') == 'Validation'))

# Further split training+test into train and test
(train_df := train_test_df.filter(pl.col('split') == 'Training'))
(test_df := train_test_df.filter(pl.col('split') == 'Test'))

DNR_ID_Site_Number,LAKE_NAME,Year,latitude,longitude,avg_secchi_depth,avg_total_phosphorus,parcel_count_within_500m,parcel_count_between_501_1600m,parcel_count_over_1600m,emv_land_mean_within_500m,emv_land_mean_between_501_1600m,emv_land_mean_over_1600m,emv_bldg_mean_within_500m,emv_bldg_mean_between_501_1600m,emv_bldg_mean_over_1600m,emv_total_mean_within_500m,emv_total_mean_between_501_1600m,emv_total_mean_over_1600m,acres_poly_mean_within_500m,acres_poly_mean_between_501_1600m,acres_poly_mean_over_1600m,acres_deed_mean_within_500m,acres_deed_mean_between_501_1600m,acres_deed_mean_over_1600m,fin_sq_ft_mean_within_500m,fin_sq_ft_mean_between_501_1600m,fin_sq_ft_mean_over_1600m,year_built_mean_within_500m,year_built_mean_between_501_1600m,year_built_mean_over_1600m,num_units_mean_within_500m,num_units_mean_between_501_1600m,num_units_mean_over_1600m,total_tax_mean_within_500m,total_tax_mean_between_501_1600m,total_tax_mean_over_1600m,…,year_built_median_over_1600m,total_tax_median_within_500m,total_tax_median_between_501_1600m,total_tax_median_over_1600m,emv_land_std_within_500m,emv_land_std_between_501_1600m,emv_land_std_over_1600m,emv_bldg_std_within_500m,emv_bldg_std_between_501_1600m,emv_bldg_std_over_1600m,emv_total_std_within_500m,emv_total_std_between_501_1600m,emv_total_std_over_1600m,acres_poly_std_within_500m,acres_poly_std_between_501_1600m,acres_poly_std_over_1600m,num_units_total_within_500m,num_units_total_between_501_1600m,num_units_total_over_1600m,basement_prop_within_500m,basement_prop_between_501_1600m,basement_prop_over_1600m,garage_prop_within_500m,garage_prop_between_501_1600m,garage_prop_over_1600m,green_acre_prop_within_500m,green_acre_prop_between_501_1600m,green_acre_prop_over_1600m,open_space_prop_within_500m,open_space_prop_between_501_1600m,open_space_prop_over_1600m,ag_preserv_prop_within_500m,ag_preserv_prop_between_501_1600m,ag_preserv_prop_over_1600m,centroid_lat,centroid_long,split
str,str,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""27062700-01""","""Northwood Lake""",2010,45.025563,-93.391715,0.98,0.1369,1875,5222,6087,68345.6,89336.844121,71123.903401,158703.573333,194060.283416,166406.177099,227049.173333,283397.127537,237530.080499,0.481685,0.537135,0.574258,0.0,0.0,0.0,1177.661867,1135.151666,1167.160013,1928.978133,1912.48717,1903.439133,0.0,0.0,0.0,3278.376,5176.414975,4174.708066,…,1962.0,3024.0,3025.0,2935.0,112070.986926,204400.048882,133033.92642,328313.70949,409851.697753,306613.611426,439743.841661,587272.905164,427788.388955,1.244839,2.085727,1.092719,0.0,0.0,0.0,0.200533,0.144006,0.158863,0.949333,0.883187,0.915065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.02944,-93.39533,"""Test"""
"""82009002-01""","""Wilmes Lake""",2007,44.927944,-92.912605,1.263636,0.080727,1255,4067,1937,88831.952191,77066.953528,120252.452246,258181.115538,232763.830834,323718.74032,347013.067729,309830.784362,443971.192566,0.0,0.0,0.0,0.471825,0.363664,0.623454,0.0,0.0,0.0,1896.645418,1860.947627,1639.568405,0.0,0.0,0.0,0.0,0.0,0.0,…,1986.0,0.0,0.0,0.0,60568.894491,107575.864077,432214.912389,90770.955605,565968.589123,1.1235e6,104814.448208,652044.332504,1.5237e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.92633,-92.936,"""Test"""
"""19034800-01""","""Valley Lake""",2014,44.71509,-93.209024,1.591667,0.07525,689,5154,1681,45012.481858,40763.484672,135513.444378,151422.641509,164708.672875,226214.039262,196435.123367,205472.157548,361727.483641,0.396967,0.235978,1.653195,0.0,0.0,0.0,1883.30479,1924.439853,3071.737656,1931.069666,1954.475747,1851.554432,0.965167,1.143772,0.996431,2233.458636,2549.304618,6912.567519,…,2000.0,2191.0,1931.0,2628.0,55939.571485,77092.183646,464504.749858,161655.087654,834787.280844,427920.47513

### Seperate Response and Predictor Variables

In [39]:
# Prepare X and y for training, test, and validation sets
# Training data
(X_train := train_df.select(feature_cols).to_pandas())
(y_train_secchi := train_df.select('avg_secchi_depth').to_pandas().values.ravel())
(y_train_phosphorus := train_df.select('avg_total_phosphorus').to_pandas().values.ravel())

# Test data
(X_test := test_df.select(feature_cols).to_pandas())
(y_test_secchi := test_df.select('avg_secchi_depth').to_pandas().values.ravel())
(y_test_phosphorus := test_df.select('avg_total_phosphorus').to_pandas().values.ravel())

# Validation data
(X_validation := validation_df.select(feature_cols).to_pandas())
(y_validation_secchi := validation_df.select('avg_secchi_depth').to_pandas().values.ravel())
(y_validation_phosphorus := validation_df.select('avg_total_phosphorus').to_pandas().values.ravel())

# Setup cross-validation
(cv_obj := KFold(n_splits=5, shuffle=True, random_state=42))

KFold(n_splits=5, random_state=42, shuffle=True)

## Step 2: Model Training - Secchi Depth

### Grid Search for CART and Random Forest

In [40]:
# CART Grid Search for Secchi Depth
cart_params = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [10, 20, 30, 50],
    'min_samples_leaf': [5, 10, 20, 30],
    'ccp_alpha': [0.0, 0.01, 0.05]
}

(cart_grid_secchi := GridSearchCV(
    DecisionTreeRegressor(random_state=42), 
    cart_params, 
    cv=cv_obj, 
    scoring='neg_mean_squared_error',
    n_jobs=-1
))

cart_grid_secchi.fit(X_train, y_train_secchi)

(cart_best_score_secchi := (-1 * cart_grid_secchi.best_score_) ** 0.5)
cart_grid_secchi.best_params_

{'ccp_alpha': 0.0,
 'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 10}

In [41]:
# Random Forest Grid Search for Secchi Depth
rf_params = {
    'n_estimators': [20, 50, 100],
    'max_depth': [5, 10, 15],
    'min_samples_split': [10, 20, 30],
    'min_samples_leaf': [5, 10, 20],
    'max_features': ['sqrt', 'log2']
}

(rf_grid_secchi := GridSearchCV(
    RandomForestRegressor(random_state=42), 
    rf_params, 
    cv=cv_obj, 
    scoring='neg_mean_squared_error',
    n_jobs=-1
))

rf_grid_secchi.fit(X_train, y_train_secchi)

(rf_best_score_secchi := (-1 * rf_grid_secchi.best_score_) ** 0.5)
rf_grid_secchi.best_params_

{'max_depth': 15,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 100}

### Compare Models on Test Set

In [42]:
# Evaluate CART on test set
(y_test_pred_cart_secchi := cart_grid_secchi.predict(X_test))
(cart_test_mse_secchi := mean_squared_error(y_test_secchi, y_test_pred_cart_secchi))
(cart_test_rmse_secchi := cart_test_mse_secchi ** 0.5)
(cart_test_r2_secchi := r2_score(y_test_secchi, y_test_pred_cart_secchi))

# Evaluate Random Forest on test set
(y_test_pred_rf_secchi := rf_grid_secchi.predict(X_test))
(rf_test_mse_secchi := mean_squared_error(y_test_secchi, y_test_pred_rf_secchi))
(rf_test_rmse_secchi := rf_test_mse_secchi ** 0.5)
(rf_test_r2_secchi := r2_score(y_test_secchi, y_test_pred_rf_secchi))

# Compare results
pd.DataFrame({
    'Model': ['CART', 'Random Forest'],
    'RMSE': [cart_test_rmse_secchi, rf_test_rmse_secchi],
    'R²': [cart_test_r2_secchi, rf_test_r2_secchi]
})

,Model,RMSE,R²
0,CART,0.912326,-0.104099
1,Random Forest,0.772506,0.208389


### Refit Best Model on Training + Test, Validate on Validation Set

In [43]:
# Combine training and test sets
(X_train_test := pd.concat([X_train, X_test]))
(y_train_test_secchi := np.concatenate([y_train_secchi, y_test_secchi]))

# Determine best model (lower RMSE is better)
(best_model_type_secchi := 'Random Forest' if rf_test_rmse_secchi < cart_test_rmse_secchi else 'CART')
(best_model_secchi := rf_grid_secchi.best_estimator_ if rf_test_rmse_secchi < cart_test_rmse_secchi else cart_grid_secchi.best_estimator_)

# Refit on combined training + test
best_model_secchi.fit(X_train_test, y_train_test_secchi)

# Evaluate on validation set
(y_val_pred_secchi := best_model_secchi.predict(X_validation))
(val_rmse_secchi := mean_squared_error(y_validation_secchi, y_val_pred_secchi) ** 0.5)
(val_r2_secchi := r2_score(y_validation_secchi, y_val_pred_secchi))

pd.DataFrame({
    'Metric': ['Best Model', 'Validation RMSE', 'Validation R²'],
    'Secchi Depth': [best_model_type_secchi, val_rmse_secchi, val_r2_secchi]
})

,Metric,Secchi Depth
0,Best Model,Random Forest
1,Validation RMSE,1.156008
2,Validation R²,0.012475


## Step 3: Model Training - Total Phosphorus

### Grid Search for CART and Random Forest

In [44]:
# CART Grid Search for Total Phosphorus
(cart_grid_phosphorus := GridSearchCV(
    DecisionTreeRegressor(random_state=42), 
    cart_params, 
    cv=cv_obj, 
    scoring='neg_mean_squared_error',
    n_jobs=-1
))

cart_grid_phosphorus.fit(X_train, y_train_phosphorus)

(cart_best_score_phosphorus := (-1 * cart_grid_phosphorus.best_score_) ** 0.5)
cart_grid_phosphorus.best_params_

{'ccp_alpha': 0.0,
 'max_depth': 7,
 'min_samples_leaf': 10,
 'min_samples_split': 30}

In [45]:
# Random Forest Grid Search for Total Phosphorus
(rf_grid_phosphorus := GridSearchCV(
    RandomForestRegressor(random_state=42), 
    rf_params, 
    cv=cv_obj, 
    scoring='neg_mean_squared_error',
    n_jobs=-1
))

rf_grid_phosphorus.fit(X_train, y_train_phosphorus)

(rf_best_score_phosphorus := (-1 * rf_grid_phosphorus.best_score_) ** 0.5)
rf_grid_phosphorus.best_params_

{'max_depth': 15,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 20}

### Compare Models on Test Set

In [46]:
# Evaluate CART on test set
(y_test_pred_cart_phosphorus := cart_grid_phosphorus.predict(X_test))
(cart_test_mse_phosphorus := mean_squared_error(y_test_phosphorus, y_test_pred_cart_phosphorus))
(cart_test_rmse_phosphorus := cart_test_mse_phosphorus ** 0.5)
(cart_test_r2_phosphorus := r2_score(y_test_phosphorus, y_test_pred_cart_phosphorus))

# Evaluate Random Forest on test set
(y_test_pred_rf_phosphorus := rf_grid_phosphorus.predict(X_test))
(rf_test_mse_phosphorus := mean_squared_error(y_test_phosphorus, y_test_pred_rf_phosphorus))
(rf_test_rmse_phosphorus := rf_test_mse_phosphorus ** 0.5)
(rf_test_r2_phosphorus := r2_score(y_test_phosphorus, y_test_pred_rf_phosphorus))

# Compare results
pd.DataFrame({
    'Model': ['CART', 'Random Forest'],
    'RMSE': [cart_test_rmse_phosphorus, rf_test_rmse_phosphorus],
    'R²': [cart_test_r2_phosphorus, rf_test_r2_phosphorus]
})

,Model,RMSE,R²
0,CART,0.083456,-1.132476
1,Random Forest,0.059895,-0.098379


### Refit Best Model on Training + Test, Validate on Validation Set

In [47]:
# Prepare combined training + test for phosphorus
(y_train_test_phosphorus := np.concatenate([y_train_phosphorus, y_test_phosphorus]))

# Determine best model (lower RMSE is better)
(best_model_type_phosphorus := 'Random Forest' if rf_test_rmse_phosphorus < cart_test_rmse_phosphorus else 'CART')
(best_model_phosphorus := rf_grid_phosphorus.best_estimator_ if rf_test_rmse_phosphorus < cart_test_rmse_phosphorus else cart_grid_phosphorus.best_estimator_)

# Refit on combined training + test
best_model_phosphorus.fit(X_train_test, y_train_test_phosphorus)

# Evaluate on validation set
(y_val_pred_phosphorus := best_model_phosphorus.predict(X_validation))
(val_rmse_phosphorus := mean_squared_error(y_validation_phosphorus, y_val_pred_phosphorus) ** 0.5)
(val_r2_phosphorus := r2_score(y_validation_phosphorus, y_val_pred_phosphorus))

pd.DataFrame({
    'Metric': ['Best Model', 'Validation RMSE', 'Validation R²'],
    'Total Phosphorus': [best_model_type_phosphorus, val_rmse_phosphorus, val_r2_phosphorus]
})

,Metric,Total Phosphorus
0,Best Model,Random Forest
1,Validation RMSE,0.067222
2,Validation R²,0.26382
